In [2]:
import pandas as pd
from utils import load_json, save_json   

In [29]:
# key case = 1
# not key case = 0

# load all files in result and create a dataframe with the following columns 
# id, result.classification, ground_truth
# save the dataframe as csv

import pandas as pd
import os

def return_binary_classification(result):
    if result == 'KEY CASE':
        return 1
    else:
        return 0

def transalte_importance_score(importance):
    if importance == '1':
        return 1
    else:
        return 0

results_dir = './results/'
results = os.listdir(results_dir)

data = []
for result in results:
    result_data = load_json(os.path.join(results_dir, result))
    data.append({
        #'id': result_data['id'],
        'classification': return_binary_classification(result_data['result']['classification']),
        'ground_truth': transalte_importance_score(result_data['importance']),
        'importance': result_data['importance'],
        'case_id': result[9:]
    })

df = pd.DataFrame(data)

In [30]:
df

,classification,ground_truth,importance,case_id
0,0,1,1,001-197098.json
1,1,0,3,001-218302.json
2,1,0,2,001-122893.json
3,0,0,3,001-217624.json
4,1,1,1,001-184525.json
...,...,...,...,...
495,0,0,3,001-221369.json
496,0,0,2,001-118738.json
497,0,0,4,001-228675.json
498,1,1,1,001-207360.json


In [17]:
errors = df[df['classification'] != df['ground_truth']]
errors  

,classification,ground_truth,importance,id
0,0,1,1,001-197098.json
1,1,0,3,001-218302.json
2,1,0,2,001-122893.json
7,1,0,2,001-146044.json
9,1,0,2,001-119055.json
...,...,...,...,...
482,1,0,2,001-201761.json
485,1,0,3,001-220885.json
487,1,0,2,001-105624.json
492,1,0,3,001-219642.json


In [28]:
e1 = errors[errors['importance'] == '1']['id'].to_list()
e2 = errors[errors['importance'] == '2']['id'].to_list()
e3 = errors[errors['importance'] == '3']['id'].to_list()
e4 = errors[errors['importance'] == '4']['id'].to_list()

len(e1), len(e2), len(e3), len(e4)

(38, 79, 54, 30)

In [25]:
list(set(e1) & set(e2) & set(e3) & set(e4))

[]

In [32]:
# Create error type column
df['error_type'] = 'correct'
df.loc[(df['classification'] == 1) & (df['ground_truth'] == 0), 'error_type'] = 'false_positive'
df.loc[(df['classification'] == 0) & (df['ground_truth'] == 1), 'error_type'] = 'false_negative'

# Analyze errors by importance level
error_by_importance = df[df['error_type'] != 'correct'].groupby(['importance', 'error_type']).size().unstack()

# Calculate error rates by importance level
total_by_importance = df.groupby('importance').size()
error_rates = error_by_importance.div(total_by_importance, axis=0) * 100

In [33]:
total_by_importance

importance
1    125
2    125
3    125
4    125
dtype: int64

In [34]:
error_rates

error_type,false_negative,false_positive
importance,,
1,30.4,NaN
2,NaN,63.2
3,NaN,43.2
4,NaN,24.0


In [35]:
# Find most commonly misclassified cases
error_cases = df[df['classification'] != df['ground_truth']]
error_summary = {
    'false_positives': error_cases[error_cases['classification'] > df['ground_truth']]['case_id'].tolist(),
    'false_negatives': error_cases[error_cases['classification'] < df['ground_truth']]['case_id'].tolist()
}
error_summary

ValueError: Can only compare identically-labeled Series objects